# Lab 06: Abstracts of Scientific Articles (1/2)
In this lab we will begin working with abstracts of scientific articles. These abstracts are obtained from [arXiv.org](http://arxiv.org), an open access repository for e-prints of articles in scientific fields maintained by Cornell University. For example, [here is a not-so-random example](https://arxiv.org/abs/1902.06034) of an article that was posted to arXiv just yesterday. In this first lab, we will begin by building a vocabulary of tokens. We will then build topic models using the python library gensim, and do some analysis over the topics obtained.

In terms of programming content, we will build on the concepts of dictionaries and regular expressions. We will introduce the Counter class, an extension of dictionaries, as well as the concept of vocabularies, which are useful for using more powerful text processing techniques. We will also give a brief overview of topic models.

In [ ]:
from datascience import *
import numpy as np
import re
import gensim

from collections import Counter

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
logging.root.level = logging.CRITICAL 

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# direct plots to appear within the cell, and set their style
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

The abstracts are in a JSON format in `abstracts.json`. Here is an excerpt from the data we'll be using.

```
{
   "1407.0001": "The topic of finding effective strategy to halt virus in complex network is of current interest. We propose an immunization strategy for seasonal epidemics that occur periodically. Based on the local information of the infection status from the previous epidemic season, the selection of vaccinated nodes is optimized gradually. The evolution of vaccinated nodes during iterations demonstrates that the immunization tends to locate in both global hubs and local hubs. We analyze the epidemic prevalence by a heterogeneous mean-field method and present numerical simulations of our model. This immunization performs superiorly to some other previously known strategies. Our work points out a new direction in immunization of seasonal epidemics.", 
```

Once again we will read this using the `json` package.

In [ ]:
import json
with open('abstracts.json') as f:
    data = json.load(f)

The JSON is in a key-value format, where the key is a unique arXiv identifier, and the value is the abstract. We are only concerned with the abstracts, so we will extract them into an array.

**In the next cell and subsequent cells, you will see code commented out. The commented out code has the same functionality as the code prior. The only difference between them is that the commented out ones are written in for loops, while the ones not commented are written using list comprehensions. If you study them, you should be able to understand why they are equivalent!**

In [ ]:
sample = 30

abstracts = [item[1] for item in data.items()]

# abstracts = []
# for item in data.items():
#     abstracts.append(item[1])
    
print("Number of documents: %d\n" % len(abstracts))
print(abstracts[sample])

This abstract is from article [arXiv:1407.0064v1](https://arxiv.org/abs/1407.0064v1) on the arXiv. From the sample document, we see that the abstracts can have various marked up text such as `\noindent`, `$y$`, or `\textit{n-inflation}`. This is LaTeX, a text markup system that allows for more convenient scientific technical writing. For example, the text `\sum_{i=1}^ni = \frac{n(n+1)}{2}` is processed to

$$\sum_{i=1}^ni = \frac{n(n+1)}{2}$$

LaTeX is widely used in academia across fields like statistics, computer science, linguistics and political science.

For our purposes, we have to remove such markup as we only want to process natural text. We can once again use regular expressions to remove the markup.

In [ ]:
# remove $...$ or $$...$$ math markup
abstracts = [re.sub(r'\$\$.*?\$\$', r'', abstract) for abstract in abstracts]
abstracts = [re.sub(r'\$.*?\$', r'', abstract) for abstract in abstracts]

# remove \emph{...}, \textit{...} type markup and replace with ...
abstracts = [re.sub(r'\\\S*?\{(.*?)\}', r'\1', abstract) for abstract in abstracts]

# remove \noindent type markup
abstracts = [re.sub(r'\\(\S*)', r'', abstract) for abstract in abstracts]

# remove {{...}}, {...}, and (...) markup, and replace with ...
abstracts = [re.sub(r'\{\{(.*?)\}\}', r'\1', abstract) for abstract in abstracts]
abstracts = [re.sub(r'\{(.*?)\}', r'\1', abstract) for abstract in abstracts]
abstracts = [re.sub(r'\((.*?)\)', r'\1', abstract) for abstract in abstracts]

# replace '-' with ' ', then remove punctuation
abstracts = [re.sub(r'-', ' ', abstract) for abstract in abstracts]
abstracts = [re.sub(r'[^\w\s]', '', abstract) for abstract in abstracts]

print(abstracts[sample])

Now, we want to further process each abstract by converting it to lower case, stripping leading and trailing white space, and then tokenizing by splitting on spaces.

In [ ]:
abstracts_tok = [abstract.lower().strip().split(' ') for abstract in abstracts]

# abstracts_tok = []
# for abstract in abstracts:
#     processed = abstract.lower().strip().split(' ')
#     abstracts_tok.append(processed)

Furthermore, we will remove tokens that have digits, have possessives or contractions, or are empty strings.

### In Class Exercise!
Write functions that will achieve that functionality described. Specifically, write
- `is_numeric(string)` which checks if `string` has any numbers
- `has_poss_contr(string)` which checks if `string` has possessives or contractions
- `empty_string(string)` which checks if `string` is an empty string
- `remove_string(string)` which checcks if `string` should be removed

In [ ]:
def is_numeric(string):
    return any(char.isdigit() for char in string)

def has_poss_contr(string):
    return '\'s' in string

def empty_string(string):
    return not string

def remove_string(string):
    return is_numeric(string) or has_poss_contr(string) or empty_string(string)

In [ ]:
# def is_numeric(string):
#     for char in string:
#         if char.isdigit():
#             return True
#     return False

# def has_poss_contr(string):
#     for i in range(len(string) - 1):
#         if string[i] == '\'' and string[i+1] == 's':
#             return True
#     return False

# def empty_string(string):
#     return string == ''

# def remove_string(string):
#     return is_numeric(string) | has_poss_contr(string) | empty_string(string)

In [ ]:
abstracts_tok = [[token for token in abstract if not remove_string(token)] for abstract in abstracts_tok]

# temp = []
# for abstract in abstracts_tok:
#     filtered = []
#     for token in abstract:
#         if not remove_string(token):
#             filtered.append(token)
#     temp.append(filtered)
# abstracts_tok = temp

Topic modelling is a powerful statistical method for discovering abstract "topics" that occur in a collection of documents. The overall intuition behind topic models is that similar documents contain similar words. In this manner, the frequency of words in a document represent the hidden semantics of the document. If two documents have similar frequently appearing words, then they are more likely to belong to a similar topic. This frequency is determined in reference to a vocabulary of all words in the collection of documents.

For example, the two documents below are similar because over the entire vocabulary of words, they contain similar frequently appearing words like "complex", "network", "epidemic".

```
{
   "1407.0001": "The topic of finding effective strategy to halt virus in complex network is of current interest. We propose an immunization strategy for seasonal epidemics that occur periodically. Based on the local information of the infection status from the previous epidemic season, the selection of vaccinated nodes is optimized gradually. The evolution of vaccinated nodes during iterations demonstrates that the immunization tends to locate in both global hubs and local hubs. We analyze the epidemic prevalence by a heterogeneous mean-field method and present numerical simulations of our model. This immunization performs superiorly to some other previously known strategies. Our work points out a new direction in immunization of seasonal epidemics.", 
```

```
  "1407.0774": "The spread of disease on complex networks has attracted widely attention in the physics community. Recent works have demonstrated that heterogeneous degree and weight distributions have a significant influence on the epidemic dynamics. In this study, a novel edge-weight based compartmental approach is developed to estimate the epidemic threshold and epidemic size (final infected density) on networks with general degree and weight distributions, and a remarkable agreement with numerics is obtained. Even in complex network with the strong heterogeneous degree and weight distributions, this approach is worked. We then propose an edge-weight based removal strategy with different biases, and find that such a strategy can effectively control the spread of epidemic when the highly weighted edges are preferentially removed, especially when the weight distribution of a network is extremely heterogenous. The theoretical results from the suggested method can accurately predict the above removal effectiveness.", 
```

To build topic models, we require the following components:
- A vocabulary of tokens that appear across all documents.
- A mapping of those tokens to a unique integer identifier, because topic model algorithms treat words by these identifiers, and not the strings themselves. For example, we represent `'epidemic'` as `word2id['epidemic'] = 50`
- The corpus, where each document in the corpus is a collection of tokens, where each token is represented by the identifier and the number of times it appears in the document. For example, in the first document above the token `'epidemic'`, which appears twice, is represented as `(50, 2)`

First, we will build a vocabulary representing the tokens that have appeared across all the abstracts we have. When doing this, we typically want to (1) remove rare words, (2) remove stop words and (3) stem/lemmatize words (which we will not do).

To do this, we can use the `Counter` class. The `Counter` is an extension of the Python dictionary, and also has key-value pairs. For the `Counter`, keys are the objects to be counted, while values are their counts.

In [ ]:
vocab = Counter()
for abstract in abstracts_tok:
    vocab.update(abstract)

print("Number of unique tokens: %d" % len(vocab))

Removing rare words helps prevent our vocabulary from being too large. Many tokens appear only a few times across all the abstracts. Keeping them in the vocabulary increases subsequent computation time. Furthermore, their presence tends not to carry much significance for a document, since they can be considered as anomalies.

We remove rare words by only keeping tokens that appear more than 25 times across all abstracts

In [ ]:
vocab = Counter(token for token in vocab.elements() if vocab[token] > 25)

# tokens = []
# for token in vocab.elements():
#     if vocab[token] > 25:
#         tokens.append(token)
# vocab = Counter(tokens)

print("Number of unique tokens: %d" % len(vocab))

Stop words are defined as very common words such as `'the'` and `'a'`. Removing stop words is important because their presence also does not carry much significance, since they appear in all kinds of texts.

We will remove stop words by removing the 200 most common tokens across all abstracts

In [ ]:
stop_words = [item[0] for item in vocab.most_common(200)]
vocab = Counter(token for token in vocab.elements() if token not in stop_words)

# stop_words = []
# for item in vocab.most_common(200):
#     stop_word = item[0]
#     stop_words.append(stop_word)
# tokens = []
# for token in vocab.elements():
#     if token not in stop_words:
#         tokens.append(token)
# vocab = Counter(tokens)

print("Number of unique tokens: %d" % len(vocab))

Now we create a mapping for tokens to unique identifiers. 

In [ ]:
id2word = {idx: pair[0] for idx, pair in enumerate(vocab.items())}
word2id = {pair[0]: idx for idx, pair in enumerate(vocab.items())}

# items = vocab.items()
# id2word = {}
# word2id = {}
# idx = 0
# for word, count in vocab.items():
#     id2word[idx] = word
#     word2id[word] = idx
#     idx += 1
    
print("Number of tokens mapped: %d" % len(id2word))
print("Identifier for 'epidemic': %d" % word2id['epidemic'])
print("Word for identifier %d: %s" % (word2id['epidemic'], id2word[word2id['epidemic']]))

Now, we will remove, for each abstract, the tokens that are not found in our vocabulary.

In [ ]:
abstracts_tok = [[token for token in abstract if token in vocab] for abstract in abstracts_tok]

# temp = []
# for abstract in abstracts_tok:
#     filtered = []
#     for token in abstract:
#         if token in vocab:
#             filtered.append(token)
#     temp.append(filtered)
# abstracts_tok = temp

Let's create the corpus. Recall that for the tokens in an abstract:

```
['statistical', 'inconsistency', 'binomial', 'likelihood', 'count', 'identified.', 'issue', 'occurs', 'when', 'response,', 'zero', 'statistically', 'inconsistent', 'erroneous', 'parameter', 'inferences', 'being', 'drawn', 'data.', 'binomial', 'likelihood', 'address', 'issue', 'resulting', 'fully', 'symmetric', 'binomial', 'likelihood', 'zero', 'counts.', 'present', 'simple', 'regression', 'example', 'ecological', 'literature', 'details', 'practical', 'application', 'likelihood', 'model.']
```
the corpus has the format
```
[(1396, 1), (1397, 1), (1398, 3), (1399, 4), (1400, 1), (1401, 1), (1402, 2), (1403, 1), (591, 1), (672, 1), (34, 2), (1404, 2), (1405, 1), (1406, 1), (1407, 1), (444, 1), (1408, 1), (1409, 1), (1410, 1), (1187, 1), (610, 1), (1411, 1), (857, 1), (1412, 1), (41, 1), (554, 1), (1413, 1), (1414, 1), (1415, 1), (1416, 1), (1205, 1), (91, 1), (945, 1)]
```

where each element is a pair containing the identifier for the token and the count of that token in just that abstract.

In [ ]:
corpus = []
for abstract in abstracts_tok:
    abstract_count = Counter(abstract)
    corpus.append([(word2id[item[0]], item[1]) for item in abstract_count.items()])

# corpus = []
# for abstract in abstracts_tok:
#     abstract_count = Counter(abstract)
#     corpus_doc = []
#     for item in abstract_count.items():
#         pair = (word2id[item[0]], item[1])
#         corpus_doc.append(pair)
#     corpus.append(corpus_doc)

print("Abstract, tokenized:\n", abstracts_tok[sample], "\n")
print("Abstract, in corpus format:\n", corpus[sample])

Now, we are ready to create our topic model!

We will use gensim, a Python library to create topic models. We will use the algorithm called latent dirichlet allocation implemented in the gensim library. 

**This step takes some time (about 4 min)**

In [ ]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

After building the topic model, we want to view the 10 topics. The topics are represented as a combination of keywords with corresponding weight on the keyword. Note that the order of these topics can change between different training runs of the topic model, since there is no ordering between topics and gensim returns them in an arbitrary order.

In [ ]:
num_topics = 10
num_words = 15
top_words = Table().with_column('word rank', np.arange(1,num_words+1))
for k in np.arange(num_topics): 
    topic = lda_model.get_topic_terms(k, num_words)
    words = [id2word[topic[i][0]] for i in np.arange(num_words)]
    probs = [topic[i][1] for i in np.arange(num_words)]
    top_words = top_words.with_column('topic %d' % k, words)
    
top_words.show()

We can compute the probability distribution for a given abstract in the `corpus`. This represents how likely it is for the abstract to belong to each topic.

In [ ]:
sample = 10
lda_model.get_document_topics(corpus[sample])

Let's represent this as a table.

In [ ]:
topic_dist = lda_model.get_document_topics(corpus[sample])
topics = [pair[0] for pair in topic_dist]
probabilities = [pair[1] for pair in topic_dist]
topic_dist_table = Table().with_columns('Topic', topics, 'Probabilities', probabilities)
topic_dist_table.show(20)
t = np.argmax(probabilities)
print("Topic with highest probability: %d (%f)" % (t, probabilities[t]))

And now as a histogram.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
fig.set_size_inches(11,4)
plt.bar(topic_dist_table.column('Topic'), topic_dist_table.column('Probabilities'), align='center', alpha=1, color='salmon')
plt.xlabel('topic')
plt.ylabel('probability')
plt.title('Per Topic Probability Distribution')
plt.show()

So it seems the document

```
  "1407.0034": "We use cosmological simulations to assess how the explosion of the first stars in supernovae (SNe) influences early cosmic history. Specifically, we investigate the impact by SNe on the host systems for Population~III (Pop~III) star formation and explore its dependence on halo environment and Pop~III progenitor mass. We then trace the evolution of the enriched gas until conditions are met to trigger second-generation star formation. To this extent, we quantify the recovery timescale, which measures the time delay between a Pop~III SN explosion and the appearance of cold, dense gas, out of which second-generation stars can form. We find that this timescale is highly sensitive to the Pop~III progenitor mass, and less so to the halo environment. For Pop~III progenitor masses, $M_{\\ast}=15{{\\,M_\\odot}}$, $25{{\\,M_\\odot}}$, and $40{{\\,M_\\odot}}$ in a halo of $5\\times10^5{{\\,M_\\odot}}$, recovery times are $\\sim10$\\,Myr, $25$\\,Myr, and $90$\\,Myr, respectively. For more massive progenitors, including those exploding in pair instability SNe, second-generation star formation is delayed significantly, for up to a Hubble time. The dependence of the recovery time on the mass of the SN progenitor is mainly due to the ionizing impact of the progenitor star. Photoionization heating increases the gas pressure and initiates a hydrodynamical response that reduces the central gas density, an effect that is stronger in more massive. The gas around lower mass Pop~III stars remains denser and hence the SN remnants cool more rapidly, facilitating the subsequent re-condensation of the gas and formation of a second generation of stars. In most cases, the second-generation stars are already metal-enriched to $\\sim2-5\\times10^{-4}{{\\rm \\,Z_\\odot}}$, thus belonging to Population~II. The recovery timescale is a key quantity governing the nature of the first galaxies, able to host low-mass, long-lived stellar systems. These in turn are the target of future deep-field campaigns with the {\\it James Webb Space Telescope}.", 
```

has the greatest likelihood to fall under the topic number with topic relating to cosmology, which matches our intuition.

## Topics for Abstracts

*Your turn!* Your task is broken down into three parts

#### 1. Label Topics
Label all the 10 topics with your interpretation of what the topics are. This may not come naturally since we are dealing with scientific articles, but do your best! Incorporate the labels by changing the names of the columns in the `top_words` table above. Also form a new table `labels` with 2 columns, where the column names are `topic_num` and `topic_label`

#### 2. Table of Topics for Abstracts
Create a function `create_topic_table(data, abstracts, corpus, lda_model)` which does the following:
- Goes through every abstract in the word-index form `corpus`, finding the most likely topic for that abstract
- Creates a table `topic_table` that has the following columns
    - `arxiv id`: the arXiv document number of each abstract
    - `topic`: the topic number of the most likely topic for each abstract
    - `label`: the topic label of that topic number, which you assigned in part 1
    - `prob`: the probability of that topic number for each abstract
    - `abstract`: a string containing the first 200 characters of the abstract
- Show the first 10 rows of the table, then return the table

You can refer to the function skeleton below.

In [ ]:
def create_topic_table(data, abstracts, corpus, lda_model):
    
    # retrieve the arXiv ids by iterating over data.items()
    
    # retrieve the topic numbers, labels and probabilities
    # for each abstract in the corpus, 
    # - find the topic distribution for the abstract
    # - get the topic number with highest probability,
    # - get the topic label associated with tthee topic number
    # - get the label for that topic
    
    # create table with
    # - column of arXiv ids
    # - column of topic numbers
    # - column of topic labels
    # - columnn of topic probabilities of that topic
    # - column of text for the abstract
    
    # show first 10 rows of the table 
    # return the table
    
    # Here is some sample code you could use to get started:
    arxiv_id = [item[0] for item in data.items()]
    
    # initialize some arrays
    probs = []
    topic = []
    label = []
    short_abstract = []
    for sample in np.arange(len(corpus)):
        topic_dist = lda_model.get_document_topics(corpus[sample])
        short_abstract.append(abstracts[sample][0:200])
        # Now find the most probable topic by calling 
        # topic_dist = lda_model.get_document_topics(corpus[sample])
        # Try using np.argmax

    table = Table().with_columns('arxiv id', arxiv_id, 'abstract', short_abstract)
    # You'll need to add the topic, label, and probability for each abstract
    # table = table.with_columns('topic', topic, 'prob', probs, 'label', label)
    return table

topic_table = create_topic_table(data, abstracts, corpus, lda_model)
topic_table.show(20)

#### 3. Discussion
Discuss the results. Choose two or three articles and describe how the most probable topic does or does not seem to accurately represent the main theme of the paper. Include your comments in a Markdown cell, with code cells added as needed to pull out particular rows of your table. Note that you can see the original scientific article a given `<arxiv id>` by visiting the web site `http://arxiv.org/abs/<arxiv id>`.  For example, [http://arxiv.org/abs/1407.0001](http://arxiv.org/abs/1407.0001) shows the first paper in the table.

Submit your lab in html and ipynb forms, as usual.